En esta primera parte analizaré la información que puedo obtener de la base de datos. La API usada es https://profit.com/es/ y en primer lugar comprobaré de que índices bursatiles puedo sacar información. 

In [235]:
!pip install yfinance


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [236]:
#Importamos lo necesario para trabajar con la API Profit.com y seteamos 
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime
import time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


Creo también el archivo tokens.env que añadiré al gitignore para que no suba a git y en el que guardaré los dos tokens que tengo de mi API. 

In [237]:
import os
from dotenv import load_dotenv

# Ruta al archivo tokens.env (asumiendo que está en la misma carpeta que el notebook)
load_dotenv("tokens.env")

# Acceder a tus tokens
api_key = os.getenv("API_KEY")
otro_token = os.getenv("API_KEY_2")  # cambia el nombre según tus claves reales

print("API Key cargada:", api_key is not None)


API Key cargada: True


La URL de la API, siguiendo los datos obtenidos de la documentación, cambia en función de los datos que queramos obtener por lo que voy a ir definiendo diferentes variables en función de los datos que quiera obtener. la información de la API está completa en: https://api.profit.com/#tag/Reference-Data

In [238]:
#Definimos la URL de la API para ver los índices que podemos sacar
URL_INDEX = f"https://api.profit.com/data-api/reference/indices?token=" + api_key
URL_INDEX


'https://api.profit.com/data-api/reference/indices?token=978336172b8c4b29960d9366896f6ef7'

In [239]:
#Creo la request para ver los índices disponibles y le meto un código de error si no funciona
response = requests.get(URL_INDEX)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code} - {response.text}")

#Vemos los índices disponibles
data


{'data': [{'ticker': 'NDX.INDX',
   'symbol': 'NDX',
   'name': 'Nasdaq 100',
   'type': 'INDEX',
   'currency': 'USD',
   'country': 'United States',
   'exchange': 'INDX'},
  {'ticker': 'GSPC.INDX',
   'symbol': 'GSPC',
   'name': 'S&P 500 Index',
   'type': 'INDEX',
   'currency': 'USD',
   'country': 'United States',
   'exchange': 'INDX'},
  {'ticker': 'DJI.INDX',
   'symbol': 'DJI',
   'name': 'Dow Jones Industrial Average',
   'type': 'INDEX',
   'currency': 'USD',
   'country': 'United States',
   'exchange': 'INDX'},
  {'ticker': 'NAS100.OANDA',
   'symbol': 'NAS100',
   'name': 'US Nas 100',
   'type': 'cfd',
   'currency': 'USD',
   'country': 'United States',
   'exchange': 'OANDA'},
  {'ticker': 'SPX500.OANDA',
   'symbol': 'SPX500',
   'name': 'US SPX 500',
   'type': 'cfd',
   'currency': 'USD',
   'country': 'United States',
   'exchange': 'OANDA'},
  {'ticker': 'US500.ICMARKETS',
   'symbol': 'US500',
   'name': 'USA S&P 500 Index',
   'type': 'cfd',
   'currency': 'US

In [240]:
#Convierto el JSON a un DataFrame para verlo mejor
df = pd.DataFrame(data['data'])
df

,ticker,symbol,name,type,currency,country,exchange
0,NDX.INDX,NDX,Nasdaq 100,INDEX,USD,United States,INDX
1,GSPC.INDX,GSPC,S&P 500 Index,INDEX,USD,United States,INDX
2,DJI.INDX,DJI,Dow Jones Industrial Average,INDEX,USD,United States,INDX
3,NAS100.OANDA,NAS100,US Nas 100,cfd,USD,United States,OANDA
4,SPX500.OANDA,SPX500,US SPX 500,cfd,USD,United States,OANDA
...,...,...,...,...,...,...,...
995,BUK350N.INDX,BUK350N,Cboe UK 350,INDEX,EUR,United Kingdom,INDX
996,BUK350P.INDX,BUK350P,Cboe UK 350,INDEX,EUR,United Kingdom,INDX
997,BUKACN.INDX,BUKACN,Cboe UK All Companies,INDEX,EUR,United Kingdom,INDX
998,BUKBISN.INDX,BUKBISN,Cboe UK Banking and Investment Services,INDEX,EUR,United Kingdom,INDX


In [241]:
#filtramos para ver solo los índices que nos interesan, en este caso el de la bolsa española
df_Spain  = df[df["country"] == "Spain"]
df_Spain

,ticker,symbol,name,type,currency,country,exchange
34,ES35.ICMARKETS,ES35,Spanish IBEX 35 Index,cfd,EUR,Spain,ICMARKETS
35,ES35.PEPPERSTONE,ES35,Spain 35 Index,cfd,EUR,Spain,PEPPERSTONE
108,IBEX.INDX,IBEX,IBEX 35 Index,INDEX,EUR,Spain,INDX
918,IBEXTR.INDX,IBEXTR,IBEX Total Return,INDEX,EUR,Spain,INDX


De los 4 tickers que tenemos de España, vamos a seleccionar el ticker "IBEX.INDX" para nuestro análisis ya que los dos primeros son cfds que tienen como archivo subyacente el propio índice. Entre los dos índices que nos ofrece, el Total Return supone una estrategia que consiste en que se hayan reinvertido todos los dividendos dados por las empresas que lo forman. Esto no es del todo válido para nuestro análisis ya que las cotizaciones de nuestras empresas las vamos a estudiar descontando los dividendos.

Ya que tenemos el índice seleccionado, vamos a ver cuales son las empresas que lo componen, para ello necesitamos la URL de las acciones y filtrar por el índice.

In [242]:
#definimos la url del índice que nos interesa para sacar el listado de empresas que lo componen, en este caso el IBEX35
URL_Empresas = f"https://api.profit.com/data-api/fundamentals/indexes/index_constituents/IBEX?token=" + api_key
URL_Empresas


'https://api.profit.com/data-api/fundamentals/indexes/index_constituents/IBEX?token=978336172b8c4b29960d9366896f6ef7'

In [243]:
response = requests.get(URL_Empresas)
if response.status_code == 200:
    data_empresas = response.json()
else:
    print(f"Error: {response.status_code} - {response.text}")
#Hacemos lo mismo que antes, convertimos el JSON a un DataFrame para verlo mejor
data_empresas

{'0': {'Ticker': 'GRF.MC',
  'Code': 'GRF',
  'Exchange': 'MC',
  'Name': 'Grifols S.A.',
  'Sector': 'Healthcare',
  'Industry': 'Drug Manufacturers - General',
  'Weight': None},
 '1': {'Ticker': 'IDR.MC',
  'Code': 'IDR',
  'Exchange': 'MC',
  'Name': 'Indra A',
  'Sector': 'Technology',
  'Industry': 'Information Technology Services',
  'Weight': None},
 '2': {'Ticker': 'CABK.MC',
  'Code': 'CABK',
  'Exchange': 'MC',
  'Name': 'Caixabank SA',
  'Sector': 'Financial Services',
  'Industry': 'Banks - Regional',
  'Weight': None},
 '3': {'Ticker': 'REP.MC',
  'Code': 'REP',
  'Exchange': 'MC',
  'Name': 'Repsol',
  'Sector': 'Energy',
  'Industry': 'Oil & Gas Integrated',
  'Weight': None},
 '4': {'Ticker': 'TEF.MC',
  'Code': 'TEF',
  'Exchange': 'MC',
  'Name': 'Telefonica',
  'Sector': 'Communication Services',
  'Industry': 'Telecom Services',
  'Weight': None},
 '5': {'Ticker': 'BKT.MC',
  'Code': 'BKT',
  'Exchange': 'MC',
  'Name': 'Bankinter',
  'Sector': 'Financial Services'

In [244]:
#Convertimos el diccionario a un DataFrame para verlo mejor. como es un diccionario con indice usaremos la primera parte de cada clave como índice
df_empresas = pd.DataFrame.from_dict(data_empresas, orient='index')
df_empresas.head(5)


,Ticker,Code,Exchange,Name,Sector,Industry,Weight
0,GRF.MC,GRF,MC,Grifols S.A.,Healthcare,Drug Manufacturers - General,None
1,IDR.MC,IDR,MC,Indra A,Technology,Information Technology Services,None
2,CABK.MC,CABK,MC,Caixabank SA,Financial Services,Banks - Regional,None
3,REP.MC,REP,MC,Repsol,Energy,Oil & Gas Integrated,None
4,TEF.MC,TEF,MC,Telefonica,Communication Services,Telecom Services,None


Ya que tenemos los datos de las empresas y sabemos utilizar la api correctamente, vamos a empezar a generar las tablas que necesitaremos para nuestro análisis que son 4, de momento las voy a guardar en csv dentro de la carpeta:

1. Empresas
Nombre
Ticker
Sector
Subsector

2. Ratios financieros
Año
Ticker
PER
ROE
EV/EBITDA
Dividend Yield
Beta
Deuda/EBITDA

3. Cotizaciones
Fecha
Ticker
Precio de cierre
Variación
Volumen

4. Dividendos
Fecha
Ticker
Importe por acción


In [245]:
df_empresas.columns

Index(['Ticker', 'Code', 'Exchange', 'Name', 'Sector', 'Industry', 'Weight'], dtype='object')

In [246]:
#Para crear la tabla empresas, primero tenemos que filtrar el DataFrame para quedarnos solo con las columnas que nos interesan. En este caso, nos quedamos con el nombre de la empresa, su ticker, sector y subsector.
df_data_empresas = df_empresas[["Name", "Ticker", "Sector", "Industry"]]
df_data_empresas.rename(columns={"Industry":"Subsector"}, inplace=True)
df_data_empresas.head(5)
df_data_empresas.to_csv("Data/01.Empresas_IBEX35.csv", index=False)

C:\Users\ganad\AppData\Local\Temp\ipykernel_6916\1435054455.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_empresas.rename(columns={"Industry":"Subsector"}, inplace=True)


In [247]:
#Para no usar la API cada vez que queramos ver los datos, cargamos de nuevo el dataframe desde el CSV
df_data_empresas = pd.read_csv("Data/01.Empresas_IBEX35.csv")
df_data_empresas

,Name,Ticker,Sector,Subsector
0,Grifols S.A.,GRF.MC,Healthcare,Drug Manufacturers - General
1,Indra A,IDR.MC,Technology,Information Technology Services
2,Caixabank SA,CABK.MC,Financial Services,Banks - Regional
3,Repsol,REP.MC,Energy,Oil & Gas Integrated
4,Telefonica,TEF.MC,Communication Services,Telecom Services
5,Bankinter,BKT.MC,Financial Services,Banks - Regional
6,Amadeus IT Group S.A.,AMS.MC,Technology,Information Technology Services
7,Mapfre,MAP.MC,Financial Services,Insurance - Diversified
8,Industria de Diseno Textil SA,ITX.MC,Consumer Cyclical,Apparel Retail
9,Banco de Sabadell S.A,SAB.MC,Financial Services,Banks - Diversified


In [350]:
#Ahora que tenemos el listado de empresas, vamos a sacar los tickers de cada una de ellas
lista_tickers = df_data_empresas["Ticker"].tolist()
lista_tickers

['GRF.MC',
 'IDR.MC',
 'CABK.MC',
 'REP.MC',
 'TEF.MC',
 'BKT.MC',
 'AMS.MC',
 'MAP.MC',
 'ITX.MC',
 'SAB.MC',
 'MTS.MC',
 'ANA.MC',
 'SAN.MC',
 'ENG.MC',
 'ACX.MC',
 'IAG.MC',
 'FER.MC',
 'IBE.MC',
 'ACS.MC',
 'BBVA.MC',
 'ELE.MC',
 'SLR.MC',
 'AENA.MC',
 'CLNX.MC',
 'LOG.MC',
 'FDR.MC',
 'COL.MC',
 'ROVI.MC',
 'MRL.MC',
 'SCYR.MC',
 'UNI.MC',
 'NTGY.MC',
 'ANE.MC',
 'RED.MC']

In [249]:
#Vamos a sacar la cotizaición de una de las empresas, por ejemplo, CABK para ver el tipo de dato que nos devuelve
ticker = "^IBEX"
cabk = yf.Ticker(ticker)
data = cabk.history(start = "2020-01-01")
#obtenemos los 5 ultimos datos
data.tail(5)






,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-04-11 00:00:00+02:00,12364.200195,12419.000000,12157.400391,12286.000000,135190900,0.0,0.0
2025-04-14 00:00:00+02:00,12540.599609,12633.799805,12419.500000,12609.799805,120617900,0.0,0.0
2025-04-15 00:00:00+02:00,12652.400391,12881.099609,12626.599609,12879.299805,124665000,0.0,0.0
2025-04-16 00:00:00+02:00,12848.500000,12942.099609,12756.099609,12942.099609,127312200,0.0,0.0
2025-04-17 00:00:00+02:00,12934.000000,12965.700195,12852.000000,12918.000000,118973300,0.0,0.0


In [306]:
#Vamos a crear un bucle para sacar la cotización de todas las empresas del IBEX35 y guardarlas en un DataFrame.
#Definimos la fecha de inicio ya que la de fin es la fecha de hoy
start_date = "2020-01-01"
#Generamos un DataFrame vacío para guardar los datos
df_cotizaciones = pd.DataFrame()
for ticker in lista_tickers:
    ticker_yahoo = yf.Ticker(ticker)
    #sacamos la cotización desde la fecha de inicio hasta hoy
    data = ticker_yahoo.history(start = start_date).reset_index()
    #le metemos el ticker como columna al DataFrame
    data["ticker"] = ticker
    #Añadimos una columna de variación porcentual de la cotización usando el método pct_change()
    data["Variación"] = data["Close"].pct_change()
    #Cambiamos el tipo de la columna "Date" a datetime y formato (YYYY/MM/DD) para que sea más fácil de trabajar
    data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
    #Concatenamos los datos de cada empresa al DataFrame vacío que hemos creado antes
    df_cotizaciones = pd.concat([df_cotizaciones, data])


#Vemos los datos que hemos sacado
df_cotizaciones

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,Variación
0,2020-01-02 00:00:00+01:00,30.970768,31.117363,30.814399,30.990313,392672,0.0,0.0,GRF.MC,NaN
1,2020-01-03 00:00:00+01:00,30.716669,31.039179,30.716669,31.019632,641127,0.0,0.0,GRF.MC,0.000946
2,2020-01-06 00:00:00+01:00,30.843717,31.166228,30.736215,31.009859,386910,0.0,0.0,GRF.MC,-0.000315
3,2020-01-07 00:00:00+01:00,31.088042,31.928522,31.088042,31.860111,2090693,0.0,0.0,GRF.MC,0.027419
4,2020-01-08 00:00:00+01:00,31.801477,31.987164,31.645109,31.830795,609008,0.0,0.0,GRF.MC,-0.000920
...,...,...,...,...,...,...,...,...,...,...
1352,2025-04-11 00:00:00+02:00,18.340000,18.719999,18.250000,18.530001,1131080,0.0,0.0,RED.MC,0.017573
1353,2025-04-14 00:00:00+02:00,18.709999,18.719999,18.459999,18.670000,685584,0.0,0.0,RED.MC,0.007555
1354,2025-04-15 00:00:00+02:00,18.790001,19.090000,18.700001,19.080000,885114,0.0,0.0,RED.MC,0.021960
1355,2025-04-16 00:00:00+02:00,19.230000,19.250000,19.000000,19.090000,876076,0.0,0.0,RED.MC,0.000524


In [308]:
df_cotizaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45754 entries, 0 to 1356
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype                        
---  ------        --------------  -----                        
 0   Date          45754 non-null  datetime64[ns, Europe/Madrid]
 1   Open          45754 non-null  float64                      
 2   High          45754 non-null  float64                      
 3   Low           45754 non-null  float64                      
 4   Close         45754 non-null  float64                      
 5   Volume        45754 non-null  int64                        
 6   Dividends     45754 non-null  float64                      
 7   Stock Splits  45754 non-null  float64                      
 8   ticker        45754 non-null  object                       
 9   Variación     45720 non-null  float64                      
dtypes: datetime64[ns, Europe/Madrid](1), float64(7), int64(1), object(1)
memory usage: 3.8+ MB


Para tener más limpio el código y posteriormente poder usarlo de forma más sencilla, voy a crear una función que haga todo lo anterior dándole una fecha de inicio y un listado de empresas

In [351]:
#Creamos la función para que nos de la cotización de un listado de empresas desde una fecha hasta el día de hoy y añada las del IBEX35
def get_cotizaciones_Ibex(tickers, start_date="2020-01-01"):
    tickers.append("^IBEX")
    df_cotizaciones = pd.DataFrame()
    for ticker in tickers:
        ticker_yahoo = yf.Ticker(ticker)
        #sacamos la cotización desde la fecha de inicio hasta hoy
        data = ticker_yahoo.history(start = start_date).reset_index()
        #le metemos el ticker como columna al DataFrame
        data["ticker"] = ticker
        data.drop_duplicates(inplace=True)
        #Añadimos una columna de variación porcentual de la cotización usando el método pct_change()
        data["Variación"] = data["Close"].pct_change()
        #Cambiamos el tipo de la columna "Date" a datetime y formato (YYYY/MM/DD) para que sea más fácil de trabajar
        data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
        #Concatenamos los datos de cada empresa al DataFrame vacío que hemos creado antes
        df_cotizaciones = pd.concat([df_cotizaciones, data])

    #Vamos a redondear a 4 dígitos la variación porcentual para que no ocupe tanto espacio y la columna de cierrre a 2 decimales
    df_cotizaciones["Variación"] = df_cotizaciones["Variación"].round(4)
    df_cotizaciones["Close"] = df_cotizaciones["Close"].round(2)
    #Eliminamos las columnas que no nos interesan, en este caso "Dividends", "Stock Splits","Open","High","Low"
    df_cotizaciones.drop(columns=["Dividends", "Stock Splits","Open","High","Low"], inplace=True)
    #Reordenamos las columnas para que quede esté el ticker la primera columna y el volumen la última
    df_cotizaciones = df_cotizaciones[["ticker","Date","Close","Volume","Variación"]]
    return df_cotizaciones.to_csv("Data/03.Cotizaciones_IBEX35.csv", index=False)


In [352]:
#Ejecutamos la función para que nos de la cotización de todas las empresas del IBEX35 desde el 2020-01-01 hasta hoy
get_cotizaciones_Ibex(lista_tickers)

In [353]:
df_cot = pd.read_csv("Data/03.Cotizaciones_IBEX35.csv")


In [354]:
df_cot["ticker"].unique()

array(['GRF.MC', 'IDR.MC', 'CABK.MC', 'REP.MC', 'TEF.MC', 'BKT.MC',
       'AMS.MC', 'MAP.MC', 'ITX.MC', 'SAB.MC', 'MTS.MC', 'ANA.MC',
       'SAN.MC', 'ENG.MC', 'ACX.MC', 'IAG.MC', 'FER.MC', 'IBE.MC',
       'ACS.MC', 'BBVA.MC', 'ELE.MC', 'SLR.MC', 'AENA.MC', 'CLNX.MC',
       'LOG.MC', 'FDR.MC', 'COL.MC', 'ROVI.MC', 'MRL.MC', 'SCYR.MC',
       'UNI.MC', 'NTGY.MC', 'ANE.MC', 'RED.MC', '^IBEX'], dtype=object)

In [355]:
#Haré un pequeño análisis de los datos para comprobar que todo ha ido bien y ver si hay algún error en los datos.
df_cot.describe(include="object")

,ticker,Date
count,47111,47111
unique,35,1357
top,GRF.MC,2025-04-17 00:00:00+02:00
freq,1357,35


In [356]:
df_cot.groupby("ticker").agg({"Close":"count"}).sort_values(by="Close", ascending=False)

,Close
ticker,
ACS.MC,1357
ACX.MC,1357
AENA.MC,1357
AMS.MC,1357
ANA.MC,1357
BBVA.MC,1357
BKT.MC,1357
CLNX.MC,1357
CABK.MC,1357


In [357]:
df_cot.isnull().sum()

ticker        0
Date          0
Close         0
Volume        0
Variación    35
dtype: int64